# Day 2 - Lab 1: AI-Generated System Design & Database Seeding

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Estimated Time:** 150 minutes

**Introduction:**
Welcome to Day 2! Today, we transition from *what* we're building to *how* we'll build it. In this lab, you will act as the lead architect for the Onboarding Tool. Your task is to use the PRD to define the data structure of the application and create a tangible database artifact that will be used for the rest of the course.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the `day1_prd.md` artifact from Day 1. This document is the primary source of truth for our project and provides the necessary context for the LLM to make intelligent design suggestions.

**Model Selection:**
Feel free to experiment with different models by changing the `model_name` in `setup_llm_client()`. Models with strong reasoning capabilities, like `gpt-4o`, `o3`, or `gemini-2.5-pro`, are excellent choices for design tasks.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the PRD from the `artifacts` directory.
- `save_artifact()`: To save the generated SQL schema and seed data.
- `clean_llm_output()`: To remove markdown fences from the generated SQL code.

In [2]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the PRD from Day 1
prd_content = load_artifact("artifacts/prd_gen.md")
if not prd_content:
    print("Warning: Could not load day1_prd.md. Lab may not function correctly.")

adr_content = load_artifact("artifacts/adr.md")
if not adr_content:
    print("Warning: Could not load ard.md. Lab may not function correctly.")

2025-11-05 15:25:00,044 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating the SQL Schema

**Task:** Use the PRD to generate a normalized SQL schema for the application.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a Database Administrator (DBA).
2.  Provide the `prd_content` as context.
3.  Ask the LLM to design a normalized SQL schema with at least two tables (e.g., `users` and `onboarding_tasks`).
4.  The output should be the raw `CREATE TABLE` statements.
5.  Save the generated SQL to `artifacts/schema.sql`.

In [7]:
# TODO: Write a prompt to generate the SQL schema from the PRD.
schema_prompt = f"""
You are an experienced Database Administrator (DBA) tasked with designing a normalized SQL schema for an AI-Powered Requirement Analyzer web application built with FastAPI and SQLModel/SQLite architecture.

CONTEXT:
Below is the complete Product Requirements Document (PRD) that describes the application you need to design a database schema for:

{prd_content}

ARCHITECTURAL CONSTRAINTS (from ADR-001):
- Backend: Single-file FastAPI application in Python
- ORM: SQLModel (provides unified ORM and Pydantic-style data validation)
- Database: SQLite for zero-configuration persistence and MVP deployment
- Background Processing: FastAPI BackgroundTasks for asynchronous AI analysis
- Deployment: Single Python file (main.py) for maximum simplicity
- Scaling: Designed for lightweight prototype with clear upgrade path to PostgreSQL

TASK:
Design a normalized SQLite database schema that supports all the functional requirements outlined in the PRD. The schema must be optimized for SQLModel ORM usage and include at least two tables following database normalization best practices.

REQUIREMENTS:
1. Create a normalized schema that eliminates data redundancy and works seamlessly with SQLModel
2. Include primary keys, foreign keys, and appropriate indexes optimized for FastAPI/SQLModel queries
3. Support all user personas mentioned in the PRD (Startup Founders, Product Managers, Development Team Leads, Consultants)
4. Handle the core workflows: problem statement capture, AI analysis, PRD generation, interactive refinement, and export/integration
5. Include tables for users, projects/PRDs, requirements, user stories, personas, chat interactions, and version control
6. Ensure data types are appropriate for SQLite and SQLModel compatibility
7. Include necessary constraints and relationships that support FastAPI's async operations
8. Support the non-functional requirements (security, scalability, data privacy) within SQLite limitations
9. Design for easy migration to PostgreSQL in future versions

KEY ENTITIES TO CONSIDER:
- Users (with different roles and OAuth 2.0/SSO authentication support)
- Projects/PRDs (with metadata, status, versions, background job tracking)
- Problem statements and input documents (with file upload support)
- Generated requirements (functional and non-functional)
- User stories with acceptance criteria
- Identified personas from AI analysis
- Chat conversations and refinement interactions (async processing)
- Export history and integrations (JSON, PDF, Word formats)
- Background job status and results
- Audit logs for compliance and debugging

SCHEMA DESIGN PRINCIPLES FOR SQLMODEL/FASTAPI:
- Use appropriate SQLite data types (TEXT, INTEGER, REAL, BLOB, DATETIME)
- Include created_at and updated_at timestamps for all entities
- Use INTEGER primary keys for better SQLModel performance (not UUIDs for SQLite)
- Implement proper foreign key relationships with CASCADE options
- Add indexes for frequently queried columns in API endpoints
- Include constraints to ensure data integrity
- Consider soft deletes for data retention compliance
- Design tables to support FastAPI's automatic request/response models
- Optimize for single-file application queries and background task updates
- Plan for future PostgreSQL migration (avoid SQLite-specific features where possible)

FASTAPI/SQLMODEL SPECIFIC CONSIDERATIONS:
- Tables should map cleanly to Pydantic models for API serialization
- Include status fields for tracking background AI processing jobs
- Design for async query patterns used by FastAPI
- Consider JSON columns for flexible metadata storage
- Include fields needed for API pagination and filtering
- Support file upload metadata and content storage

OUTPUT FORMAT:
Provide ONLY the raw CREATE TABLE statements in SQLite syntax optimized for SQLModel usage. Do not include explanatory text, comments, or markdown formatting. Ensure proper SQL syntax with semicolons after each statement.

Generate a comprehensive schema that supports the full functionality described in the PRD while adhering to the FastAPI + SQLModel + SQLite architectural decisions."""

print("--- Generating SQLite Schema ---")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, "artifacts/schema.sql", overwrite=True)
    print(f"\n✅ SQLite schema saved to artifacts/schema.sql")
else:
    print("❌ Skipping schema generation because PRD content is missing.")

--- Generating SQLite Schema ---
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    email TEXT NOT NULL UNIQUE,
    full_name TEXT,
    hashed_password TEXT,
    oauth_provider TEXT,
    oauth_id TEXT,
    role TEXT NOT NULL,
    is_active INTEGER NOT NULL DEFAULT 1,
    created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE prds (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    title TEXT NOT NULL,
    description TEXT,
    status TEXT NOT NULL,
    current_version INTEGER NOT NULL DEFAULT 1,
    created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY(user_id) REFERENCES users(id) ON DELETE CASCADE
);

CREATE INDEX idx_prds_user_id ON prds(user_id);

CREATE TABLE prd_versions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    prd_id INTEGER NOT NULL,
    version_number INTEGER NOT NULL,
    tit

### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Task:** Prompt the LLM to generate realistic seed data that conforms to the schema you just created.

**Instructions:**
1.  Create a new prompt.
2.  Provide both the `prd_content` and the `cleaned_schema` as context.
3.  Instruct the LLM to generate 5-10 realistic `INSERT` statements for your tables.
4.  The data should be relevant to a new hire onboarding tool (e.g., sample user names and task titles like "Complete HR Paperwork").
5.  Save the generated `INSERT` statements to `artifacts/seed_data.sql`.

In [9]:
# TODO: Write a prompt to generate realistic seed data.
cleaned_schema_content = load_artifact("artifacts/schema.sql")
if not cleaned_schema_content:
    print("Warning: Could not load schema. Lab may not function correctly.")

seed_data_prompt = f"""
You are a Database Administrator tasked with creating realistic seed data for an AI-Powered Requirement Analyzer web application. This application helps users convert problem statements into comprehensive Product Requirements Documents (PRDs) using AI analysis.

CONTEXT:
Below is the complete database schema you need to populate with realistic test data:

{cleaned_schema_content}

APPLICATION CONTEXT (from PRD):
{prd_content}

TASK:

CONTEXT:
Below is the complete database schema you need to populate with realistic test data:

{cleaned_schema}

APPLICATION CONTEXT (from PRD):
{prd_content}

TASK:
Generate realistic INSERT statements that populate the database with sample data representing a functioning AI-Powered Requirement Analyzer application. The seed data should demonstrate typical usage scenarios and support testing of all major application features.

REQUIREMENTS:
1. Create data for at least 5-10 realistic users representing different personas (Startup Founders, Product Managers, Development Team Leads, Consultants)
2. Include 3-5 sample PRD projects in various stages (draft, in-progress, completed)
3. Generate realistic problem statements, requirements, user stories, and personas for each PRD
4. Include sample chat interactions showing AI analysis and user refinement
5. Add background job records showing AI processing status
6. Include export history and audit logs for realistic application usage
7. Ensure all foreign key relationships are properly maintained
8. Use realistic names, email addresses, project titles, and technical requirements
9. Include timestamps that show realistic project progression over time
10. Add sample file uploads and document metadata

DATA GUIDELINES:
- Use realistic but fictional user names and email addresses
- Create PRD projects for common software applications (mobile apps, web platforms, SaaS tools)
- Generate meaningful requirement descriptions and user stories
- Include both functional and non-functional requirements
- Show realistic AI-generated personas with detailed characteristics
- Create chat conversations that demonstrate typical user-AI interactions
- Include various PRD statuses: draft, analysis_in_progress, ready_for_review, completed
- Add realistic job processing states: pending, running, completed, failed
- Use appropriate priority levels: high, medium, low
- Include different export formats: JSON, PDF, Word
- Show integration attempts with external tools

SAMPLE DATA THEMES:
- E-commerce mobile app for small businesses
- Project management SaaS platform
- Healthcare patient portal
- Educational learning management system
- Financial budgeting and tracking tool

OUTPUT FORMAT:
Provide ONLY the raw INSERT statements in SQLite syntax. Do not include explanatory text, comments, or markdown formatting. Ensure proper SQL syntax with semicolons after each statement. Use single quotes for text values and proper date formats (YYYY-MM-DD HH:MM:SS) for DATETIME fields.

Start with users table, then prds, then related tables in logical dependency order to avoid foreign key constraint violations.

Generate comprehensive seed data that demonstrates the full functionality of the AI-Powered Requirement Analyzer application."""

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data
    save_artifact(cleaned_seed_data, 'artifacts/seed_data.sql', overwrite=True)
    print(f"\n✅ Seed data saved to artifacts/seed_data.sql")
else:
    print("❌ Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---
INSERT INTO conversation (id, title, current_prd_version_id, status, metadata, created_at, updated_at) VALUES
(1, 'E-commerce App for Artisans', 2, 'active', '{"user_id": "user_sarah_101", "persona": "Startup Founder"}', '2025-11-10 09:15:00', '2025-11-11 14:30:00'),
(2, 'SaaS Project Management Tool', 3, 'active', '{"user_id": "user_mike_202", "persona": "Product Manager"}', '2025-11-12 11:00:00', '2025-11-12 16:45:00'),
(3, 'Healthcare Patient Portal', NULL, 'active', '{"user_id": "user_david_404", "persona": "Consultant"}', '2025-11-13 10:05:00', '2025-11-13 10:07:00'),
(4, 'Personal Finance Budgeting App', 4, 'archived', '{"user_id": "user_jessica_303", "persona": "Development Team Lead"}', '2025-11-08 14:20:00', '2025-11-08 17:00:00');

INSERT INTO message (id, conversation_id, sender, message_type, content, metadata, status, created_at, updated_at) VALUES
(1, 1, 'user', 'text', 'I want to build a mobile app for local artisans to sell their handmade go

### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Task:** This is a critical technical step. You will write a Python script to execute the generated SQL files, creating a live `onboarding.db` file that your application will use.

**Instructions:**
1.  Complete the `create_database` function below.
2.  The function should first connect to (and thus create) a SQLite database file named `artifacts/onboarding.db`.
3.  It should then open and execute the `schema.sql` file to create the tables.
4.  Finally, it should open and execute the `seed_data.sql` file to populate the tables.
5.  Use a `try...finally` block to ensure the database connection is always closed, even if an error occurs.

> **Hint:** The `try...finally` block is a crucial Python pattern. The code in the `finally` block will run whether the `try` block succeeds or fails, making it the perfect place to ensure resources like database connections are always closed.

**Expected Quality:** A physical `onboarding.db` file in your `artifacts` folder. This is a tangible asset that proves your design is valid and provides a concrete foundation for backend development.

In [ ]:
import os
import sqlite3
import sys

# Set up the project root path correctly
current_dir = os.getcwd()
if "220372-AG-AISOFTDEV-Team-3-PromptPioneers" in current_dir:
    project_root = current_dir
else:
    project_root = os.path.join(current_dir, "220372-AG-AISOFTDEV-Team-3-PromptPioneers")

# Add to path if needed
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import load_artifact

def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return
    
    conn = None
    try:
        # TODO: Connect to the SQLite database. This will create the file if it doesn't exist.
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"✅ Successfully connected to database at {db_path}")

        # TODO: Read the content of the schema file using load_artifact.
        print("📖 Reading schema file...")
        schema_sql = load_artifact("artifacts/schema.sql")
        if not schema_sql:
            print("❌ Failed to load schema file")
            return
        
        # TODO: Execute the schema SQL script.
        # Hint: Use cursor.executescript() for multi-statement SQL strings.
        print("🏗️  Creating database tables...")
        cursor.executescript(schema_sql)
        print("✅ Tables created successfully.")

        # TODO: Check if the seed data file exists. If it does, load and execute it.
        if os.path.exists(seed_path):
            print("📊 Loading seed data...")
            seed_sql = load_artifact("artifacts/seed_data.sql")
            if seed_sql:
                print("🌱 Inserting seed data...")
                cursor.executescript(seed_sql)
                print("✅ Seed data inserted successfully.")
            else:
                print("⚠️  Failed to load seed data file")
        else:
            print("⚠️  Seed data file not found, skipping...")

        # TODO: Commit the changes to the database.
        conn.commit()
        print("💾 Database changes committed successfully.")
        
        # Verify the database was created properly
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        print(f"📋 Created {len(tables)} tables: {[table[0] for table in tables]}")
        
    except sqlite3.Error as e:
        print(f"❌ Database error: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
    finally:
        # TODO: Ensure the connection is closed if it was opened.
        if conn:
            conn.close()
            print("🔒 Database connection closed.")

# Define file paths
db_file = os.path.join(project_root, "artifacts", "prd-database.db")
schema_file = os.path.join(project_root, "artifacts", "schema.sql")
seed_file = os.path.join(project_root, "artifacts", "seed_data.sql")

print(f"🎯 Target database path: {db_file}")
print(f"📄 Schema file path: {schema_file}")
print(f"🌱 Seed file path: {seed_file}")

# Verify files exist before proceeding
print(f"📁 Project root: {project_root}")
print(f"📋 Schema file exists: {os.path.exists(schema_file)}")
print(f"📋 Seed file exists: {os.path.exists(seed_file)}")

# Clear/delete existing database if it exists
if os.path.exists(db_file):
    os.remove(db_file)
    print(f"🗑️  Removed existing database file: {db_file}")
else:
    print("ℹ️  No existing database file found.")

# Execute the function
create_database(db_file, schema_file, seed_file)

🎯 Target database path: c:\Users\labadmin\artifacts\prd-database.db
📄 Schema file path: c:\Users\labadmin\artifacts\schema.sql
🌱 Seed file path: c:\Users\labadmin\artifacts\seed_data.sql
ℹ️  No existing database file found.
Error: Schema file not found at c:\Users\labadmin\artifacts\schema.sql


## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.